In [5]:
from ipywidgets import interactive, widgets, fixed
from surfer import Brain as surface
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import minmax_scale

import os, sys
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# spectrome:
from spectrome.brain import Brain
from spectrome.utils import functions, path

# wilson cowan model:
module_path = '/Users/xxie/lab/model_inference/'
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(os.path.join(module_path, 'model_inference'))
    
from model_inference.models.WilsonCowanModel import WilsonCowanModel
from model_inference.functions import utils

from spectrum import dpss

plt.style.use('bmh')

In [6]:
%gui qt

# pysurfer variables:
subject_id = 'fsaverage'
hemi = ['lh', 'rh']
surf = 'white'

# parcellation:
hemi_side = 'lh'
aparc_file = os.path.join(os.environ['SUBJECTS_DIR'],
                         subject_id, 'label',
                         hemi_side + '.aparc.annot')
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

In [33]:
# function for viewing canonical networks:
def get_fc_values(fc_df, labels, fc_name):
    # get our data ready in both hemispheres
    fc_network = fc_df.loc[fc_name].values
    lh_cort = minmax_scale(fc_network[0:34])
    rh_cort = minmax_scale(fc_network[34:68])

    # for pysurfer requirements
    lh_pad = np.insert(lh_cort, [0, 3], [0, 0])
    rh_pad = np.insert(rh_cort, [0, 3], [0, 0])

    lh_fc = lh_pad[labels]
    rh_fc = rh_pad[labels]

    fc_brain = surface(
        subject_id,
        "both",
        surf,
        background="white",
        alpha=1,
        title="Canonical Networks",
    )
    fc_brain.hide_colorbar()
    fc_brain.add_data(lh_fc, hemi="lh", thresh=0.15, colormap=plt.cm.autumn_r, remove_existing=True)
    fc_brain.add_data(rh_fc, hemi="rh", thresh=0.15, colormap=plt.cm.autumn_r, remove_existing=True)
    #fc_brain.scale_data_colormap(color_fmin, color_fmid, color_fmax, transparent=False)
    return lh_fc, rh_fc


# function for simulating and plotting oscillator model:
duration = 2000
fs = 0.2
times = np.linspace(0, duration, int(duration*fs))
nfft = int(4096/8)

[tapers, eigen] = dpss(len(times), 7)
band, frequencies = utils.get_frequencies(
    nfft,
    200,
    fmin=2,
    fmax=45
)

def simulate_neural_mass(tauE, tauI, P):
    nmm = WilsonCowanModel(times, fs = fs*1000, nfft = nfft)
    ts, psd = nmm.simulate([tauE, tauI, P])
    axs = plt.figure(figsize=(10,4), constrained_layout=True).subplots(1,2)
    axs[0].plot(times[int(fs*1000):], ts)
    axs[0].title.set_text('Time course')
    axs[0].set_xlabel('Time (ms)')
    axs[1].plot(frequencies, np.log10(psd))
    axs[1].title.set_text('Frequency Spectrum')
    axs[1].set_xlabel('Frequency (Hz)')
    axs[1].set_ylabel('Magnitude')
    return ts, psd

In [5]:
fc_names = [
    "Visual",
    "Limbic",
    "Default",
    "Somatomotor",
    "Frontoparietal",
    "Ventral_Attention",
    "Dorsal_Attention",
]
# Load Pablo's canonical networks in DK atlas:
fc_dk = np.load("/Users/xxie/lab/complex_laplacian/data/com_dk.npy", allow_pickle=True).item()
fc_dk_normalized = pd.read_csv("/Users/xxie/lab/complex_laplacian/data/DK_dictionary_normalized.csv").set_index(
    "Unnamed: 0"
)

- Introduction:
     - structure function problem
     - modalities of data
     - connectivity
     - mean field models
     
 - DWI advances
 - NMM brief explanation and results
 - complex laplacian
 - spectral graph model

<p style = "text-align:center; font-size:150%;">Model Based Analysis of Multimodal Neuroimaging: From Neural Masses to Spectral Graph Theory</p>
<p style = "text-align:center; font-size:70%;"><em>Thesis Defense</em></p>
<p style = "text-align:center; font-size:70%;"><em>Presented to the Weill Cornell Graduate School of Medical Sciences</em></p>
<p style = "text-align:center; font-size:70%;"><em>In Fulfillment of the Requirements for the Degree of Doctor of Philosophy</em></p>

<center>
<img src = "figures/presentation/whole_brain.png" alt = "intro" style = "align:center;width:700px;height:400px"/>
</center>

<p style = "text-align:center; font-size:70%;">Xi He Xie</p>
<p style = "text-align:center; font-size:70%;">Department of Neuroscience, Weill Cornell Medicine</p>
<p style = "text-align:center; font-size:70%;">Visiting Graduate, University of California San Francisco</p>
<p style = "text-align:center; font-size:70%;">Nov 8th, 2021</p>

What's so special about the human brain?
---

 - Humans have the remarkable ability to invent symbol systems such as the arabic numeric system or the alphabet.

<center>
    <img src = "figures/presentation/lost_data.png" alt = "messy_data" width = "900" style = "align:center" />
</center>

<aside class="notes">
Before I start talking in jargons and data, let's think for a second. I'm sure everyone here had days like this guy, we are constantly reading papers, crunching data, doing numerical analysis, and it can get extremely messy if you don't practice good organizational routines. Despite all that, our brains are still efficiently taking these symbols, putting them into context, and processes the complicated information... if you had enough coffee
</aside>

#### Exercise:

 - What is so special about human brain functionality that allows it to acquire these cultural tools? (Language, reading, arithmetics) Let's examine the two possible hypothesis:

 1) Cultural functionality is acquired by expanded cortical plasticity unique to humans.
     

 2) Humans have evolved new specialized circuitry, each providing new cognitive functions.

<aside class="notes">
On one extreme, you can have (1), on the other you can have (2). So very quickly, by a show of hands, who agrees with 1? la la la... who agrees with (2)?

 - Human brians accomodate a broad range of new functions through learning.
    
 - At the extreme, the brain's architecture exerts little to no contraints on the range of competence we can acquire.
    
 - This may also suggest individual brain's implementation of cultural abilities is highly variable across individuals.
    
 - Humans experienced evolutionary pressure to develop brain circuitry for understanding symbols?
    
 - But reading was invented ~5000 years ago, and artihmetics even more recent!
</aside>

 - Primates have prior brain functions and structures that were eventually adapted and efficiently used by humans.
 
<!-- <center>
    <img src = "figures/presentation/xkcd_surgery.png" alt = "xkcd" width = "1000" style = "align:center" />
<center/>
<aside class="notes">
So this is from xkcd, and it's true there's an xkcd comic for everything. The brain's structure itself stays the same, but when our ancestors acquire new tools like symbols in arithmetics and reading, their brains adapted what was already available in primate brains and expand it's capabilities like a software update.
</aside> -->

### The structure-function problem in neuroscience:
The brain's structure at various spatial scales is critical for determining its function.

<b>My thesis' interest in particular: Finding the relationship between the brain's <em>structural wiring</em> and the <em>functional</em> patterns of neural activity.</b>
 - Consider the brain to be a macroscopic network of nodes and edges:
     - Identifiable gray matter brain regions of interest (ROIs)
     - Long range connections between brain regions via white matter fiber bundles

<center>
    <img src = "figures/presentation/brain_nodes_edges.png" alt = "braingraph" width = "400" style = "align:right"/>
</center>

Now we have a problem:
--- 
<center>
    <img src = "figures/presentation/xkcd_brain.png" alt = "messy_data" width = "1000" style = "align:center" />
</center>
<p style = "text-align:center; font-size:75%;"> xkcd-1163</p>
 
 - We want to look at the interesting stuff going on inside a living human brain...
 - What are some non-invasive methods of recording data from a living human brain?

#### We can measure brain data non-invasively in humans with ...

# functional data:
### 1) Human Magnetoencephalography (MEG) frequency spectrum:
<center>
    <img src = "figures/presentation/meg_spectrum_example.png" alt = "actualmeg" width = "1200" style = "align:center"/>
</center>
<aside class="notes">
The first functional human brain data we collect is called MEG. Where we have our subjects sit inside an expensive insulated room under an even more expensive helmet. The helmet is cooled to near absolute zero to cool down the SQUIDS (superconducting quantum interference devices), these sensors can detect subtle changes in magnetic field as induced by evoked currents from the neurons. Anyone remember the right hand rule from physics class?
    
The result from each sensor is then mapped to our brain regions of interest, but we don't like looking at it over time, because all we would see are squiggly oscillatory lines. So we decompose these oscillations into combinations of sin/cos waves of various frequencies and power, and plot them here in what we call a power spectrum. Humans have very replicable and similar power spectrums, where we have idenfiable delta, theta, alpha, beta, gamma waves. The alpha wave usually stands out the most, if everyone closed their eyse right now, the 10 Hz signal in the posterior regions of your brain would just light up! And beta band is often mentioned in motor tasks with beta suppresion at the lateral motor cortex
</aside>

<center>
    <img src = "figures/presentation/fmri_modalities.png" alt="fmri" width="1300" style="align:center" />
</center>

<aside class="notes">
Structural: The hydrogen molecules in our bodies contains 1 positive charge (protons), like the earth spinning on its axis with magnetic poles, each spinning charged molecule is also like a tiny magnet that spins around it's own axis. MRI uses magnetic fields and radio waves to measure locations of these molecules. These protons are all in random directions, the MRI machine uses a magnet that ranges from 1-7 Teslas to align all our protons in specific directions. For reference, the earth's magnetic field is ~0.6e-4T. The scanners produce a magnetic field that's 60,000 times stronger! 
fMRI: In our brains, we have billions of neurons, when a lot of these neurons begin to fire, the body rushes in oxygen to help supply energy. This is called a hemodynamic response, and what we measure is called blood oxygen level dependent signal (BOLD). 
DWI: The brain is connected by long distance projections called axonal fiber tracts. We specifically track the movement of water molecules inside the brain. And build a gradient of water diffusion. 
</aside>

### Structural data: diffusion weighted imaging
<center>
    <img src = "figures/presentation/tract_viz.gif" alt = "dti" width = "1200" style = "align:center"/>
</center>

# Organizing high dimensional data via connectivity - Anatomical

#### Quantify all the traced white matter streamlines between all brain regions into a matrix (Structural connectivity)

<center>
<img src = "figures/presentation/SC-example.png" alt = "SCmatrix" width = "1000" style = "align:center"/>
</center>
<aside class="notes">
Now we align both the white matter fibers and our anatomical MRI scans, from each of these ROIs, we can start counting the number of fibers going to each of the other connected regions, and summarize all the connections in this connectivity matrix. The brain is what we like to call a small world network, where there are dense local connections for specialized function, and long range connections for integration of function with other hubs of connections. Just the inter- and intra- hemispheric connections you see here, dense within hemisphere, sparse across hemispheres.
</aside>

# Connectivity - Functional

Network neuroscience described in 3 dimensions by Bassett, Zurn & Gold's review in 2018:

<center>
<img src = "figures/presentation/dani_dimensions.png" alt = "dims" style = "align:center;width:800px;height:650px"/>
</center>

## Functional Connectivity

<center>
<img src = "figures/presentation/fc_metrics.png" alt = "FC" style = "align:center;width:950px;height:500px"/>
</center>

## Simpson's Paradox:

<center>
<img src = "figures/chapter1/simpson.png" alt = "simpson" width = "1000" style = "align:center"/>
</center>

<aside class="notes">
We can use a very simple example to highlight the threat of confounding, a scenario where an unobserved interaction can either cancel out an effect or induce spurious effects. Imagine there are two brain states, and one is associated with high activity of neuron A and low activity of neuron B, while another is associated with low activity of neuron A and high activity of neuron B. If we do not know the brain state (confound) one may conclude that A has a negative influence on B. But if we know the brain states, we then see that A increases the activity of B.
</aside>

## Be careful with causality:

 - Causality implies cause and effect. 
     - if a neuron influences another neuron, then perturbing the first would affect the second. (same for brain regions).
     - However, it's always possible to constsruct a common input which will produce the same effect on a neuron that we believe is influenced by another neuron.
     - Similarly, if there's no causal influence between A and B, it's always possible to use an addititonal neuron C to change A and B.

 - With our sensors surrounding the skull, we can never have a clear idea of all the confounds in our recordings. 

 - Yet the lots of prominent publications are still claiming changes in FC or model parameters are revealing mechanistic links ("connectivity") between neuron populations.

 - The best we can say is the joint statistical distributions show potential biomarkers that helps classify certain groups of people at a particular epoch of time. (But even then...)

# Modeling FC with Mean Field Theory in Neuroscience

Mean field theory originates from studies of magnetism and fluid dynamics, where scholars needed a few macroscopic variables to describe the collective behavior of individual units.

The framework for application of mean field theory is dynamical systems, an example is Einstein's famous law of motion:

$$ \frac{dV}{dt} = \frac{F}{m} $$

But instead in neuroscience, we are computing the average firing rate over a population (neural mass) of excitatory or inhibitory neurons:

$$ \frac{dE}{dt} = -D(E) + \Psi (E) + \epsilon (t)$$

<aside class="notes">
Mean field models extention to whole brain network models... 
</aside>

#### Wilson & Cowan neural mass model:

$$\tau_e \frac{dE_{j}(t)}{dt} = -E_{j}(t) + (1 - r_e E_{j}(t)) \, S_e(c_1 E_{j}(t) - c_2 I_{j}(t) + P) + \epsilon_{j} (t)$$

$$\tau_i \frac{dI_{j}(t)}{dt} = -I_{j}(t) + (1 - r_i I_{j}(t)) \, S_i(c_3 E_{j}(t) - c_4 I_{j}(t)) + \epsilon_{j} (t)$$

### Parameter control of dynamics and Hopf bifurcation:

In [34]:
interactive(
    simulate_neural_mass,
    tauE = widgets.IntSlider(value=8,min=1,max=20,step=1,description=r'$\tau_e$',continuous_update=False,),
    tauI = widgets.IntSlider(value=8,min=1,max=20,step=1,description=r'$\tau_i$',continuous_update=False,),
    P = widgets.FloatSlider(value=1.25,min=0.5,max=2,step=0.05,description='Driving force',continuous_update=False,),
)

interactive(children=(IntSlider(value=8, continuous_update=False, description='$\\tau_e$', max=20, min=1), Int…

<center>
<img src = "figures/chapter3/oscillator_unit.png" alt = "oscillator_unit" style="align:center;width:680px;height:750px;"/>
</center>

### Can NMM capture the brain's broadband power spectrum?

###  Inferring model parameters with MCMC sampling:

<center>
<img src = "figures/chapter3/oscillator_full_10hz.png" alt = "oscillator_fullfit" style = "align:center;width:800px;height:820px"/>
</center>

<aside class="notes">
Most probable distribution of values for parameters of interest.
</aside>

### What if we aim for periodic frequency spectrum only?

<center>
<img src = "figures/chapter3/oscillator_periodic_10hz.png" alt = "oscillator_periodicfit" style = "align:center;width:800px;height:800px"/>
</center>

## Exhaustive grid search yields similar results:

<center>
<img src = "figures/chapter3/gridsearch_fullfit.png" alt = "gridsearch" width = "1500" style = "align:center"/>
</center>

<aside - add transition with identifiability issue

## Neural mass model extensions to the whole brain:

$$ \frac{dE}{dt} = -D(E) + \Psi (E) + \mathbf{A}E(t-\tau_d) + \epsilon (t)$$

<aside class="notes">
    how ridiculous is it that noise drives deviation away from limit cycle activity.
</aside>

#### Wilson-Cowan model:

$$ \tau_e \frac{dE_{j}(t)}{dt} = -E_{j}(t) + (1 - r_e E_{j}(t)) \, S_e(c_1 E_{j}(t) - c_2 I_{j}(t) + c_5 \sum_{k} A_{jk} E_{k}(t - \tau_d^k) + P) + \epsilon_{j} (t) $$

$$ \tau_i \frac{dI_{j}(t)}{dt} = -I_{j}(t) + (1 - r_i I_{j}(t)) \, S_i(c_3 E_{j}(t) - c_4 I_{j}(t)) + \epsilon_{j} (t) $$

<center>
<img src = "figures/chapter3/p_behavior.png" alt = "p_behavior" style = "align:center;width:680px;height:945px"/>
</center>

<aside class="notes">
    Talk about how it's mainly the delay and noise driving the dynamics they see. Talk about how now it's 68 x 68 / 2 number of elements to optimize for, and they are second order statistics. Talk about the spectra, can these network models capture region-wise frequency spectrum?
</aside>

#### NMMs are difficult to invert:

 - Sensitive to initial conditions.
 - Delayed networks require "history" and there is no framework for establishing this.
 - High number of parameters.
 - Integration of dynamic systems is time and resource consuming.
 - Step-wise integration with noise requires adaptive time steps.
 - Transitions betweeen quiet, stable, and limit cycle regimes create local minimas.

Phase-plane portrait of the Wilson-Cowan NMM reviewed by M. Breakspear:

<center>
<img src = "figures/chapter1/wc-pp.png" alt = "phaseplane" style = "align:center;width:1200px;height:300px"/>
</center>

#### Tricks used to avoid theses problems:

 - Simplify the model (even more).
 - Identical parameters at every node of the network.
 - Choose a subset of global parameters for inference. 
 - Choose initial conditions so that it is near a handpicked optimal solution or Hopf bifurcation point.
 - Conditional minimization of subsets of parameters.
 - Wrap NMM solution with another layer of model.

### Can a network NMM capture whole brain dynamics without these tricks?

### Does inverting models to FC translate well to capture the brain's frequency spectrum?

### Global coupling posterior simulated spectra:

<center>
<img src = "figures/chapter3/individual_c5_spectra.png" alt = "c5spectra" width = "1500" style = "align:center"/>
</center>

### All parameter posterior simulated spectra:

<center>
<img src = "figures/chapter3/individual_hopf_spectra.png" alt = "hopfspectra" width = "1500" style = "align:center"/>
</center>

### Take aways:

 - Neural mass models (NMMs) are nonlinear dynamical systems describing mean field activity of neuron populations.
 - NMMs have been heavily used to analyze whole brain functional data.
 - Second order statistics like functional connectivity cannot describe causal effects due to confounding. 
 - NMM inversion not easily extendable to the whole brain scale.
     - Identifiability issues of inferred parameters.
     - Sensitivity to initial conditions and history.
     - Non-convex optimization problem.
 - It is no surprise that deep learing has overtaken theoretical models in classification.

<p style = "text-align:center; font-size:150%;">But the joint dynamics of many neurons is still interesting!</p>

<p style = "text-align:center; font-size:100%;">Low dimensional projections of brain recordings can reveal invariant patterns or biomarkers!</p>

Canonical Functional Networks
---
 - Obtained from resting-state fMRI recordings
 - Consistently observed functional networks in both resting-state fMRI and MEG
 - There are seven of these prominent, most frequently visited functional networks identified by neuroimaging studies.

In [ ]:
interactive(
    get_fc_values,
    fc_df=fixed(fc_dk_normalized),
    labels=fixed(labels),
    fc_name=widgets.RadioButtons(
        options=fc_names, value="Limbic", description="Select canonical network"
    ),
)

## Measuring brain structure and function:

A reminder of what we have ...
- Resting-state functional recordings from MEG, fMRI functional networks

 - Anatomical magnetic resonance images (MRIs) to co-register MEG coordinates to "head" coordinates

 - Parcellated brain regions from MRI (68 cortical regions and 18 subcortical regions in our atlas of choice)

 - Diffusion weighted imaging (DTI) for anatomical wiring architecture via tractography

 - Hard working graduate students!

### How do canonical functional networks arise from structural networks?

We have a new framework combining connectivity matrices and distance matrices that results in a complex structural connectome (Laplacian), on which we do an eigen decomposition analysis with only 2 global parameters, coupling strength $\alpha$ and wave number $k$:

<center>
<img src = "figures/presentation/lap_overview.png" alt = "workflow" style = "align:center;width:1200px;height:390px"/>
</center>

<aside class="notes">
READ THE SLIDE

Remember the rotating tractogram I showed before? And we extracted structural connetivitty matrix by counting white matter connections? That's the yellow and red matrix here. But we also want to obtain the distance of all our tracts and summarize them the same way in this blue and white matrix, calling it the distance adjacency matrix. The upper triangle here is scaled by alpha, the global coupling term, indicating how much of our signal comes from connections rather than local dynamics, the second parameter wave number is paired with the lower triangle, the wave number is a spatial frequency describing the amount of distance a signal in the brain travels per oscillation. With these 2 parameters, we can obtain this next matrix called the complex Laplacian. This matrix describes the characteristic signal spread patterns in a network with eigenmodes. Now imagine you are a signal in the brain, when you travel from region to region, it's not random, you must follow the brain's structural wiring, and this matrix tells you how these wirings would guide your signal. And if we project these patterns onto the brains, we can start comparing these eigenmodes to our functional networks!
</aside>

Our goal is to relate the eigenmodes of the complex structural connectome to the canonical functional networks.

Delays become phases in the frequency domain, so we define a complex connectivity matrix as a function of $\omega$: $\pmb{C}^*(\omega) = \{ c_{j,k} exp(-j\omega \pmb{\tau_{j,k}^v}) \}$, then a normalized complex connectivity matrix at any given $\omega$ is:
$$ \pmb{C}(\omega) = diag(\frac{1}{\sqrt{\pmb{deg}}}) \pmb{C}^*(\omega) $$

Define the Laplacian: $ L(\omega,\tau^v) = I - \alpha C(\omega,\tau^v)$, and $L(\omega, \tau^v)$ is an 86x86 complex matrix because we parcellated 86 brain regions.

Eigen decompose the Laplacian: $L(\alpha,k) = U(\alpha,k)\Lambda(\alpha,k)U^H(\alpha,k)$

We believe the eigenmodes $U(\alpha,k)$ are a substrate of the brain's white matter connections on which functional activity plays out.

$U(\alpha,k)$ is a vector whose numbers represent the magnitude of power in each parcellated brain region.

## What do these eigenmodes look like?

We compare our findings with the conventional non-complex structural connectome.

<center>
<img src = "figures/presentation/eigmode_examples2.png" alt = "eig_examples" width = "1300" style = "align:center"/>
</center>

Eigenmodes of the complex Laplacian exhibit spatial patterns of fMRI canonical networks.

<center>
<img src = "figures/presentation/fig3_rescaled_compiled.png" alt = "fc_eig" style = "align:center;width:890px;height:850px"/>
</center>

<center>
<img src = "figures/presentation/cl_scatter.png" alt = "fc_scatter" width = "1100"style = "align:center"/>
</center>

#### Spatial pattern matching of complex Laplacian outperforms real Laplacian:

<center>
<img src = "figures/presentation/pearson_residual.png" alt = "pearson" width = "1600" style = "align:center"/>
</center>

### Group Level Comparison:

<center>
<img src = "figures/presentation/group_pearson.png" alt = "pearson" width = "1600" style = "align:center"/>
</center>

### Next step: Examine the frequency spectrum of eigenmodes
<center>
<img src = "figures/presentation/SGM_1.png" alt = "SGM1" width = "1200" style ="align:center"/>
</center>

<p style = "text-align:center; font-size:120%;"> Examine MEG Frequency Spectrum with a Parsimonious Structure-Function Model </p>
<center>
<img src = "figures/presentation/SGM1_model.png" alt = "sg1" width = "600" style ="align:center"/>
</center>

and the structural eigenmodes act as spatial filters!
<center>
    <img src = "figures/presentation/intro_pysurfer.png" alt = "eigenmodes" style = "align:center;width:500px;height:150px"/> 
</center>

Evaluating the eigen decomposition of the complex graph Laplacian: $L(\omega) = U(\omega)\Lambda(\omega)U^H(\omega)$, our spectral graph model is defined as:
$$ X(\omega) = \sum_{i} \frac{u_i(\omega) u_i^H (\omega)}{j\omega + \frac{1}{\tau_C} \lambda_i F_e(\omega)} H_{local}(\omega) \boldsymbol{P}(\omega) $$

## A quick to compute linear hierarchical model:

<center>
<img src = "figures/presentation/mcmc_animated.gif" alt = "mcmc_gif" width = "700" style ="align:center"/>
</center>

### Highly probable parameters that reproduce MEG spectrum

<center>
<img src = "figures/presentation/SGM_mcmc_params.png" alt = "mcmc" width = "1200" style ="align:center"/>
</center>

Group level comparison:
<center>
<img src = "figures/presentation/SGM_ave_spectrum.png" alt = "sg7" width = "1000" style ="align:center"/>
</center>

Spatial Patterns in Alpha-band:

<center>
<img src = "figures/presentation/SGM_alpha_spatial.png" alt = "sg7" width = "1000" style ="align:center"/>
</center>

Spatial Patterns in Beta-band:

<center>
<img src = "figures/presentation/SGM_beta_spatial.png" alt = "sg7" width = "1000" style ="align:center"/>
</center>

Spectral graph model performance:
<center>
    <img src = "figures/presentation/SGM_spatial_violin.png" alt = "model_comparison" width = "1200" style = "align:center"/>
</center>

## Take-aways

 - Modeling second order statistics of population mean signal is not mechanistically interesting.
     - Data driven clusters, frequency spectra, and their spatial distributions are much more informative.
 - NMM parameter inference is time consuming, sensitive to initial conditions, and faces identifiability issues.
 - The joint statistics of non-invasive brain recordings show promising patterns for further research.
 - We extracted canonical functional patterns of brain activity via complex-valued structural eigenmodes.
 - MEG Alpha and beta band activity can be mapped to individual complex-valued structural eigenmodes.
 - Our linear hierarchical model captures BOTH frequency patterns as well as spatial patterns of brain activity.

## SGM in the near future:

 - More specified local dynamics.
 - Tau PET data modeling.
 - Infant development spectral maturation.
 
## Tractography advancements:

# Thank you slides...